In [1]:
# %%
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch
from numpy import genfromtxt
torch.set_default_dtype(torch.float64)

In [2]:
# train
x = genfromtxt('data/mdata_train.txt', delimiter=',')
X_train = x.T
y_train = genfromtxt('data/mdata_train_l.txt',delimiter=',')

x_test = genfromtxt('data/mdata_test.txt', delimiter=',')
X_test = x_test.T
y_test = genfromtxt('data/mdata_test_l.txt',delimiter=',')

# Convert to torch
X_train = torch.tensor(X_train, dtype=torch.float64)
y_train = torch.tensor(y_train == 1, dtype=torch.long).unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float64)
y_test = torch.tensor(y_test == 1, dtype=torch.long).unsqueeze(1)

# Move data to GPU
X_train, y_train = X_train.cuda(), y_train.cuda()
X_test, y_test = X_test.cuda(), y_test.cuda()

# Prepare train input
xinp_train = torch.cat([torch.ones(X_train.shape[0], 1, dtype=X_train.dtype, device=X_train.device), X_train], dim=-1)
xinp_test = torch.cat([torch.ones(X_test.shape[0], 1, dtype=X_test.dtype, device=X_test.device), X_test], dim=-1)

In [3]:
from tensor.layers import TensorTrainLayer
from tensor.bregman import KLDivBregman, AutogradBregman, BinaryKLDivBregman

N = 2
r = 2
p = X_train.shape[1]+1
#C = y_train.shape[1]

# Define Bregman function
layer = TensorTrainLayer(N, r, p, output_shape=1).cuda()
y_pred = layer(xinp_train)
w = 1/y_pred.std().item()
del y_pred
bf = BinaryKLDivBregman(w=w)

def convergence_criterion(y_pred, y_true):
    accuracy = (((w*y_pred).sigmoid() > 0.5).long() == y_true).float().mean().item()
    print("Accuracy:", accuracy)
    return accuracy > 0.82

In [4]:
with torch.inference_mode():
    layer.tensor_network.accumulating_swipe(xinp_train, y_train, bf, batch_size=64, lr=1.0, convergence_criterion=convergence_criterion, orthonormalize=False, method='exact', eps=1e-5, verbose=True, num_swipes=10)

Left to right pass (A1):   0%|          | 0/2129 [00:00<?, ?it/s]

Left loss (A1): 1.1968429600293327
Accuracy: 0.7699542045593262


Left to right pass (A2):   0%|          | 0/2129 [00:00<?, ?it/s]

Left loss (A2): 0.798898693073401
Accuracy: 0.47732946276664734


Right to left pass (A2):   0%|          | 0/2129 [00:00<?, ?it/s]

...

Left loss (A2): 9.294161843156646
Accuracy: 0.7595219016075134


Right to left pass (A2):   0%|          | 0/2129 [00:00<?, ?it/s]

Right loss (A2): 6.642509705744195
Accuracy: 0.7592576146125793


Right to left pass (A1):   0%|          | 0/2129 [00:00<?, ?it/s]

Right loss (A1): 6.649811259163539
Accuracy: 0.4265703558921814


In [5]:
# Calculate accuracy on train set
y_pred_train = layer(xinp_train)
accuracy_train = (((w*y_pred_train).sigmoid() > 0.5).long() == y_train).float().mean().item()
print('Train Acc:', accuracy_train)

Train Acc: 0.4265703558921814


In [6]:
# Calculate accuracy on test set
y_pred_test = layer(xinp_test)
accuracy_test = (((w*y_pred_test).sigmoid() > 0.5).long() == y_test).float().mean().item()
print('Test Acc:', accuracy_test)

Test Acc: 0.5607324838638306


In [7]:
layer.tensor_network.nodes

[TensorNode(name=A1, shape=torch.Size([51, 2]), labels=['p', 'r2']),
 TensorNode(name=A2, shape=torch.Size([2, 51]), labels=['r2', 'p']),
 TensorNode(name=X1, shape=torch.Size([15346, 51]), labels=['s', 'p']),
 TensorNode(name=X2, shape=torch.Size([15346, 51]), labels=['s', 'p'])]

In [3]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, balanced_accuracy_score

# Convert PyTorch tensors to NumPy arrays
X_train_np = X_train.cpu().numpy()
y_train_np = y_train.cpu().numpy()
X_test_np = X_test.cpu().numpy()
y_test_np = y_test.cpu().numpy()

# Train an XGBoost classifier
xgb_clf = xgb.XGBClassifier(eval_metric='logloss')
xgb_clf.fit(X_train_np, y_train_np)

# Predict on the test set
y_pred_test_xgb = xgb_clf.predict(X_test_np)

# Calculate accuracy
accuracy_test_xgb = balanced_accuracy_score(y_test_np, y_pred_test_xgb)
print('XGBoost Test Accuracy:', accuracy_test_xgb)

XGBoost Test Accuracy: 0.9130282124122588
